In [ ]:
import pickle
import os
import sys
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve
import matplotlib.pyplot as plt

# --- 상수 정의 및 경로 설정 ---
DATA_DIR = '../data' 
UTILS_DIR = '../src' 
MIN_MER, MAX_MER = 8, 13

# --- 유틸리티 임포트 경로 설정 ---
utils_path = os.path.abspath(os.path.join(os.getcwd(), UTILS_DIR))

if utils_path not in sys.path:
    sys.path.append(utils_path)

try:
    from utils import get_kmers, check_response, emit_metrics
    print(f"✅ utils 모듈 임포트 완료. 탐색 경로: {utils_path}")
except ImportError as e:
    print(f"❌ 오류: utils 모듈 임포트 실패. 경로를 확인하세요: {e}")
    sys.exit(1)

# 테스트 데이터셋 파일 이름 정의
TEST_DATASETS = {
        "vh": 
            {
            "human": "test_human_VH_shuf_no_dupli_aligned_10k.txt", "diverse": "diverse_5_human_VH_biophi_and_test_seqs.txt",       
            "mouse": "mouse_VH_cleaned_aligned_10k.txt", "rhesus": "rhesus_VH_cleaned_aligned_10k.txt",      
            "pssm": "rd_pssm_human_VH_random_10k.txt",
            },
        "vkappa":  
            {
            "human": "test_human_VKappa_shuf_no_dupli_aligned_10k.txt", "diverse": "diverse_2_5_human_VKappa_biophi_and_test_seqs.txt", 
            "mouse": "mouse_VKappa_cleaned_aligned_10k.txt", "rhesus": "rhesus_VKappa_cleaned_aligned_10k.txt",  
            "pssm": "rd_pssm_human_VKappa_random_10k.txt",
            },
        "vlambda": 
            {
            "human": "test_human_VLambda_shuf_no_dupli_aligned_10k.txt", "diverse": "diverse_2_5_VLambda_biophi_and_test_seqs.txt",      
            "mouse": "mouse_VLambda_cleaned_aligned_all.txt", "rhesus": "rhesus_VLambda_cleaned_aligned_10k.txt", 
            "pssm": "rd_pssm_human_VLambda_random_10k.txt",
            },
        }

# --- 헬퍼 함수 정의 (변경 없음) ---

def read_txt(fname: str) -> list:
    """지정된 경로의 텍스트 파일에서 시퀀스를 읽어와 '-' 문자를 제거합니다."""
    seqs = []
    if os.path.exists(fname):
        with open(fname, 'r') as f:
            lines = f.readlines()
            for line in lines:
                seqs.append(line.strip().replace('-', ''))
        return seqs
    else:
        print(f"⚠️ 경고: 파일이 존재하지 않습니다: {fname}")
        return []

def load_kmer_pools(data_dir: str, min_mer: int, max_mer: int) -> tuple:
    """지정된 K-mer 범위에 대해 Pickle 파일에서 K-mer 풀 데이터를 로드합니다."""
    pools_human, pools_paired_human, pools_paired_mouse = {}, {}, {}
    for mer in range(min_mer, max_mer):
        try:
            pkl_human = os.path.join(data_dir, f'human_{mer}mer.dump')
            with open(pkl_human, 'rb') as f: pools_human[mer] = pickle.load(f)
            pkl_paired_human = os.path.join(data_dir, f'oas_paired_human_{mer}mer.dump')
            with open(pkl_paired_human, 'rb') as f: pools_paired_human[mer] = pickle.load(f)
            pkl_paired_mouse = os.path.join(data_dir, f'oas_paired_mouse_{mer}mer.dump')
            with open(pkl_paired_mouse, 'rb') as f: pools_paired_mouse[mer] = pickle.load(f)
        except FileNotFoundError as e:
            print(f"❌ K-mer 풀 로드 오류 (mer={mer}): 파일이 없습니다: {e}")
            raise 
    return pools_human, pools_paired_human, pools_paired_mouse

def calc_scores(sequences: list, pool_pos: list, pool_neg: list, label: int) -> tuple:
    """주어진 시퀀스 리스트에 대해 K-mer 풀을 사용하여 예측 점수를 계산합니다."""
    pred, true = [], []
    if not sequences: return pred, true
    assert len(sequences) > 0

    for sequence in sequences:
        scores, total_response, total_kmers_count = {}, 0, 0
        for mer in range(MIN_MER, MAX_MER):
            kmers = get_kmers(sequence, mer, mer)
            pool_mer_pos = [item[mer] for item in pool_pos if mer in item]
            pool_mer_neg = [item[mer] for item in pool_neg if mer in item]
            subscores, subresponse = check_response(kmers, mer, pool_mer_pos, pool_mer_neg, scores)
            total_response += subresponse
            total_kmers_count += len(kmers)
            scores = subscores
        final_score = total_response / max(1, total_kmers_count)
        pred.append(final_score)
        true.append(label)
    return pred, true

# --- classification 함수: 파일 저장 로직 제거 및 인라인 출력으로 수정 ---

def classification(pred: list, label: list, task_name: str, dataset_type: str):
    """
    예측 점수와 참 레이블을 기반으로 ROC 곡선 및 임계값별 성능(MCC, F1)을 플로팅하고 
    Jupyter Notebook에 바로 출력합니다.
    """
    
    # 1. 공통 지표 계산
    auroc = roc_auc_score(label, pred)
    auprc = average_precision_score(label, pred)

    # 2. ROC Curve 데이터 생성 및 플롯
    fpr, tpr, _ = roc_curve(label, pred)
    
    plt.figure(figsize=(7, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUROC = {auroc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title(f'ROC Curve: {dataset_type} - {task_name}', fontsize=14)
    plt.legend(loc='lower right', fontsize=10)
    plt.tight_layout()
    # plt.savefig() 호출을 제거합니다.
    plt.close() # 다음 셀 실행을 위해 figure를 닫습니다.

    # 3. MCC와 F1-Score vs. Threshold 데이터 생성 및 플롯
    intervals = np.arange(0, 1, 0.01) 
    mccs, f1s = [], []
    
    for interval in intervals:
        fpred = [1 if i >= interval else 0 for i in pred]
        metric = emit_metrics(fpred, label) 
        mccs.append(metric[2]) # MCC
        f1s.append(metric[1])  # F1-Score

    plt.figure(figsize=(7, 6))
    plt.plot(intervals, mccs, label='MCC', color='blue', lw=2)
    plt.plot(intervals, f1s, label='F1-Score', color='green', lw=2, linestyle='--')
    
    best_mcc_index = np.argmax(mccs)
    best_mcc = mccs[best_mcc_index]
    best_threshold = intervals[best_mcc_index]
    
    plt.scatter(best_threshold, best_mcc, color='red', marker='o', s=50, zorder=5,
                label=f'Optimal MCC: {best_mcc:.4f} @ Thr {best_threshold:.2f}')
    
    plt.xlabel('Threshold', fontsize=12)
    plt.ylabel('Metric Score', fontsize=12)
    plt.title(f'Performance vs. Threshold: {dataset_type} - {task_name}', fontsize=14)
    plt.ylim([0, 1])
    plt.legend(loc='lower left', fontsize=10)
    plt.grid(axis='y', linestyle='--')
    plt.tight_layout()
    # plt.savefig() 호출을 제거합니다.
    plt.close() # 다음 셀 실행을 위해 figure를 닫습니다.

    print(f"✅ {task_name} 분석 결과: AUROC={auroc:.4f}, AUPRC={auprc:.4f}, Max MCC={best_mcc:.4f} (그래프 출력 완료)")


def run_benchmark(dataset_type: str, data_dir: str):
    """K-mer 풀을 로드하고 지정된 데이터셋에 대한 벤치마크를 실행합니다."""
    
    print("--- K-mer 풀 로딩 시작 ---")
    pools_human, pools_paired_human, pools_paired_mouse = load_kmer_pools(
        data_dir=data_dir, 
        min_mer=MIN_MER, 
        max_mer=MAX_MER
    )
    print("✅ K-mer 풀 로드 완료.")

    # 양성 및 음성 풀 설정
    pool_pos = [pools_human, pools_paired_human]
    pool_neg = [pools_paired_mouse]
    
    # --- 데이터셋 로드 ---
    base_path = os.path.join(data_dir, 'abnativ', dataset_type)
    
    diverse_seqs = read_txt(os.path.join(base_path, TEST_DATASETS[dataset_type]['diverse']))
    mouse_seqs = read_txt(os.path.join(base_path, TEST_DATASETS[dataset_type]['mouse']))
    rhesus_seqs = read_txt(os.path.join(base_path, TEST_DATASETS[dataset_type]['rhesus']))
    pssm_seqs = read_txt(os.path.join(base_path, TEST_DATASETS[dataset_type]['pssm']))
    human_seqs = read_txt(os.path.join(base_path, TEST_DATASETS[dataset_type]['human']))
    
    # --- 점수 계산 ---
    print(f"\n--- 데이터셋 '{dataset_type}' 점수 계산 시작 ---")

    diverse_pred, diverse_true = calc_scores(diverse_seqs, pool_pos=pool_pos, pool_neg=pool_neg, label=1)
    human_pred, human_true = calc_scores(human_seqs, pool_pos=pool_pos, pool_neg=pool_neg, label=1)
    mouse_pred, mouse_true = calc_scores(mouse_seqs, pool_pos=pool_pos, pool_neg=pool_neg, label=0)
    rhesus_pred, rhesus_true = calc_scores(rhesus_seqs, pool_pos=pool_pos, pool_neg=pool_neg, label=0)
    pssm_pred, pssm_true = calc_scores(pssm_seqs, pool_pos=pool_pos, pool_neg=pool_neg, label=0)

    # --- 태스크 조합 ---
    tasks = {
        'human_mouse': {'pred': human_pred + mouse_pred, 'true': human_true + mouse_true},
        'human_pssm': {'pred': human_pred + pssm_pred, 'true': human_true + pssm_true},
        'human_rhesus': {'pred': human_pred + rhesus_pred, 'true': human_true + rhesus_true},
        'diverse_human_mouse': {'pred': diverse_pred + mouse_pred, 'true': diverse_true + mouse_true},
        'diverse_human_pssm': {'pred': diverse_pred + pssm_pred, 'true': diverse_true + pssm_true},
        'diverse_human_rhesus': {'pred': diverse_pred + rhesus_pred, 'true': diverse_true + rhesus_true},
    }

    # --- 결과 출력 (그래프 생성) ---
    print("\n--- 분류 지표 시각화 시작 ---")
    for k, v in tasks.items():
        task_name, pred, true = k, v['pred'], v['true']
        
        if true:
            # 수정된 classification 함수 호출 (그래프 생성 및 인라인 출력)
            classification(pred, true, task_name=task_name, dataset_type=dataset_type)
        else:
            print(f"⚠️ 태스크 '{task_name}': 데이터 부족으로 시각화를 건너뜁니다.")


if __name__ == '__main__':
    ABNATIV_DATASET_TYPE = 'vkappa'  # vh, vkappa, vlambda 중 선택 가능


    try:
        run_benchmark(ABNATIV_DATASET_TYPE, data_dir=DATA_DIR)
    except Exception as e:
        print(f"\n🔥 벤치마크 실행 중 치명적인 오류 발생: {e}")
        print("K-mer 풀 파일이 'data' 디렉터리 내에 올바르게 위치하는지 확인하십시오.")

✅ utils 모듈 임포트 완료. 탐색 경로: /mnt/c4a1c56d-0589-48cd-9717-f6c33a86fa3b/Tools/ImmunoSeq/src
--- K-mer 풀 로딩 시작 ---
✅ K-mer 풀 로드 완료.

--- 데이터셋 'vkappa' 점수 계산 시작 ---
